# Field Analysis

> Latest stats are from 2021, either because scimago stopped collecting data, or newer data is deemed unreliable regarding citations

In [1]:
import numpy as np
import pandas as pd

In [2]:
yearly_df = pd.read_csv("journal_record.csv")

In [3]:
df = pd.read_csv("journal.csv").merge(
    yearly_df.loc[lambda _df: _df["year"] == 2021],
    left_on="sourceid",
    right_on="journal__sourceid",
)

In [4]:
area_df = pd.read_csv("journal_area.csv")

In [5]:
def get_cats(df):
    _cs = df["categories"].str.split("; ")
    idk = "sourceid"
    _df = pd.DataFrame(
        {"catbase": _cs.sum(), idk: np.repeat(df[idk].values, _cs.str.len())}
    )
    return pd.concat([_df, _df["catbase"].str.extract("(.*) \((.*)\)")], axis=1).assign(
        field=lambda df: np.where(df.loc[:, 0].isna(), df["catbase"], df.loc[:, 0])
    ).rename(columns={1: "Q"}).loc[:, ["field", "Q", idk]]

In [6]:
cat_base = get_cats(df.head(10000)).merge(df)

In [7]:
field_pivot = (
    cat_base.fillna("no Q")
    .pivot_table(index="field", columns="Q", values="sourceid", aggfunc="count")
    .fillna(0)
    .assign(s=lambda df: df.sum(axis=1))
    .sort_values("s", ascending=False)
    .loc[:, lambda df: df.columns.str.startswith("Q")]
)
field_pivot.head(15).style.background_gradient(axis=1).set_caption(
    "Count values by fields"
)

Q,Q1,Q2,Q3,Q4
field,,,,
Medicine (miscellaneous),389.000000,426.000000,374.000000,307.000000
Sociology and Political Science,197.000000,97.000000,58.000000,50.000000
Mechanical Engineering,114.000000,87.000000,64.000000,68.000000
"Geography, Planning and Development",98.000000,86.000000,75.000000,74.000000
Electrical and Electronic Engineering,96.000000,71.000000,60.000000,66.000000
History,111.000000,77.000000,63.000000,63.000000
"Ecology, Evolution, Behavior and Systematics",99.000000,92.000000,82.000000,39.000000
Economics and Econometrics,127.000000,87.000000,48.000000,35.000000
Condensed Matter Physics,75.000000,79.000000,73.000000,57.000000


In [8]:
def draw_table(df):
    return (
        df.pivot_table(index="field", columns="Q")
        .loc[field_pivot.head(15).index]
        .loc[:, lambda df: df.isna().mean() < 0.5]
        .style.background_gradient(axis=0)
    )

In [9]:
(
    cat_base.groupby(["field", "Q"])[["h_index", "journal_rating", "total_docs_3years"]]
    .mean()
    .pipe(draw_table)
    .set_caption("Mean values of fields")
)

In [10]:
def gini(s):
    vc = s.value_counts(normalize=True)
    diffs = np.abs(vc.values.reshape(-1, 1) - vc.values.reshape(1, -1))
    return diffs.sum() / (2 * vc.shape[0] ** 2 * vc.mean())

def top5(s):
    return s.value_counts(normalize=True).head(5).sum()

## Concentration metrics by fields

In [11]:
cat_base.groupby(["field", "Q"])[["country", "publisher"]].agg([gini, top5]).pipe(draw_table).set_caption("Concentration metrics by fields")